# Sistemas Inteligentes

## Procura Informada


## Conteúdos

##### Procura em grafo
* Heurísticas
* Procura com heurísticas
    * Procura sôfrega (greedy) em grafo
    * A* em grafo
* Admissibilidade de uma heurística
* Consistência de uma heurística
* Exemplos de problemas
    * Pacman grave como pastilhas
    * Problema do Puzzle8
* Exercícios

## Introdução
Nesta aula vamos ver mais alguns aspetos relacionados com a resolução de problemas de acordo com o Paradigma do Espaço de Estados, usando a implementação disponibilizada pelo módulo `searchPlus.py`, introduzido nas aulas anteriores. Vamos hoje começar por ver como poderemos utilizar os dois algoritmos de procura standard, na sua versão em grafo.

In [20]:
from searchPlus import *

## Heurísticas

Vamos adicionar valores heurísticos ao grafo abstracto que temos usado, em que mudamos o custo do arco I -> B para 6.
A heurística de um estado é a estimativa da distância mais curta desse estado a um estado objectivo.
Os valores heurísticos são os seguintes:
I: 7, A: 6 B: 2 C: 4 D: 2 F: 0
e correspondem ao método h1 na classe do Problema que redefinimos a seguir

<img src="grafoXX.png" alt="Drawing" style="width: 300px;"/>

In [21]:
class ProblemaGrafoHs(Problem) :
    grafo = {'I':{'A':2,'B':6},
             'A':{'C':2,'D':4,'I':2},
             'B':{'D':1,'F':5,'I':6},
             'C':{},
             'D':{'C':3,'F':2},
             'F':{}}

    def __init__(self,initial = 'I', final = 'F') :
        super().__init__(initial,final)
        
    def actions(self,estado) :
        sucessores = self.grafo[estado].keys()  # métodos keys() devolve a lista das chaves do dicionário
        accoes = list(map(lambda x : "ir de {} para {}".format(estado,x),sucessores))
        return accoes

    def result(self, estado, accao) :
        """Assume-se que uma acção é da forma 'ir de X para Y'
        """
        return accao.split()[-1]
    
    def path_cost(self, c, state1, action, state2):
        return c + self.grafo[state1][state2]

    def h1(self,no) : 
        """Uma heurística é uma função de um estado.
        Nesta implementação, é uma função do estado associado ao nó
        (objecto da classe Node) fornecido como argumento.
        """
        h = {'I':7, 'A':6,'B':2,'C':4,'D':2,'F':0}
        return h[no.state]

In [53]:
p=ProblemaGrafoHs()

Se quisermos saber qual o valor heurístico de um estado, ele só é possível no contexto de um problema e é preciso encapsular o estado numa instância de **Node** (classe definida em *searchPlus.py*)

In [23]:
print("h1 de", 'A'," =", p.h1(Node('A')))
print("h1 de", 'F'," =", p.h1(Node('F')))

h1 de A  = 6
h1 de F  = 0


Se quisermos listar os valores heurísticos de todos os estados

In [24]:
for estado in p.grafo.keys():
    print("h1 de", estado," =", p.h1(Node(estado)))

h1 de I  = 7
h1 de A  = 6
h1 de B  = 2
h1 de C  = 4
h1 de D  = 2
h1 de F  = 0


## Procura heurística

Vamos aplicar os algoritmos de procura heurística: o Greedy Melhor Primeiro e o A*

### Greedy Melhor Primeiro (grafo)

Na Procura Sôfrega (Greedy) preferimos sempre expandir o estado de menor valor heurístico. Na prática a implementação em Python faz uso do mesmo método que o custo uniforme, que é o ***best_first_graph_search()***, que relembramos usa um fila de prioridades com base no valor heurístico. Esta versão é a em grafo, i.e., os ramos com estados já expandidos nunca são colocados na fronteira. Também são colocados dois ramos que acabam no mesmo estado, i.e., os limites da fronteira nunca podem ter estados repetidos.

```python
def best_first_graph_search(problem, f):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    f = memoize(f, 'f')
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node
    frontier = PriorityQueue(min, f)
    frontier.append(node)
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                incumbent = frontier[child]
                if f(child) < f(incumbent):
                    del frontier[incumbent]
                    frontier.append(child)
    return None
```

Enquanto o custo uniforme utiliza o custo como prioridade, ignorando o segundo argumento do método de cima, a procura sôfrega é exactamente equivalente ao método de cima. A função heurística terá que ser passada como argumento quando for usada.

```python
 greedy_best_first_graph_search = best_first_graph_search
```

Vamos executar o Melhor Primeiro (Greedy Best First), versão em grafo, para o grafo abstracto.
Notem que estamos a passar como argumento a função heurística h1.

Notem que esta parte do código lida particularmente com a questão da procura ser em grafo:

```python
for child in node.expand(problem):
    if child.state not in explored and child not in frontier:
        frontier.append(child)
    elif child in frontier:
        incumbent = frontier[child]
        if f(child) < f(incumbent):
            del frontier[incumbent]
            frontier.append(child)
```

Se o estado do nó (ponta do ramo) não tiver sido ainda expandido e se um nó com o mesmo estado (ramo com a mesma ponta) não fizer parte da fronteira, então é adicionado à fronteira, mas se já lá estiver um ramo com o mesmo estado ele nunca é adicionado porque dois nós com o mesmo estado (ponta) terão o mesmo valor heurístico e o novo nó nunca será adicionado.

In [25]:
res_gbfs = greedy_best_first_graph_search(p,p.h1)
print(res_gbfs.solution())
print(res_gbfs.path_cost)

['ir de I para B', 'ir de B para F']
11


Vejam que o Melhor Primeiro dá-nos uma solução que não é a melhor. Não garante o óptimo como se confirma com a invocação do custo uniforme, esse sim um algoritmo optimal.

In [27]:
res_best = uniform_cost_search(p)
print(res_best.solution())
print(res_best.path_cost)

['ir de I para A', 'ir de A para D', 'ir de D para F']
8


### A* (grafo)

O A* é independente dos problemas e das funções heurísticas e recebe-os como argumentos. Só temos a sua versão em grafo que invoca o ***best_first_graph_search()*** mas em que a função de prioridade é uma função lambda que dado um nó soma o custo do caminho total até ao estado inicial com o valor da função heurística do nó (respetivo estado). Para o cálculo da função heurística basta o nó, porque através dele acede-se ao custo do caminho até à origem (estado inicial) sendo o nó também o argumento da função heurística.

Notem que o uso do memoize serve para não se recalcular os valores heurísticos dos estados se já foram calculados antes, é um cache. Embora seja uma procura em grafo, é preciso recalcular as funções heurísticas no caso do estado do nó ainda não tiver sido expandido e for estado (folha ou ponta) de um nó na fronteira. É tanto mais relevante quanto maior os custos computacionais do cálculo dos valores heurísticos.

```python
def astar_search(problem, h=None):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    return best_first_graph_search(problem, lambda n: n.path_cost + h(n))
```

Vamos executar o algoritmo A* na sua versão em grafo. Só existe a versão em grafo em *searchPlus.py*.
Notem que a solução devolvida até é a de menor custo. Será que a heurística h1 é consistente?

In [28]:
res_astar = astar_search(p,p.h1)
print(res_astar.solution())
print(res_astar.path_cost)

['ir de I para A', 'ir de A para D', 'ir de D para F']
8


### Admissibilidade de uma função heurística

**Uma função heurística é considerada admissível se nunca sobreestima a sua menor distância ao objectivo.** Notem que isso quer dizer que para qualquer estado, **o valor heurístico é sempre menor ou igual à menor distância desse estado ao objectivo**.
A admissibilidade é principalmente importante para o algoritmos A* na sua versão não optimizada (em árvore), que não está implementada no *searchPlus.py*.
**Uma heurística que seja admissível garante a solução óptima no A* (árvore).**

Para verificar se é admissível iremos calcular os custos das melhores soluções de cada estado para o estado final, utilizando o algoritmo de Custo Uniforme que relembramos é optimal calculando sempre os menores custos. Esses custos irão ser comparados com os respectivos valores heurísticos.

In [29]:
for estado in p.grafo.keys():
    p = ProblemaGrafoHs(estado)
    resultado = uniform_cost_search(p)
    if resultado is None:
        print("sem solução a partir de", estado)
    else:
        print("Menor distância de", estado,"ao objectivo",p.goal, "=",resultado.path_cost)
        print("h("+estado+") =",p.h1(Node(estado)))

Menor distância de I ao objectivo F = 8
h(I) = 7
Menor distância de A ao objectivo F = 6
h(A) = 6
Menor distância de B ao objectivo F = 3
h(B) = 2
sem solução a partir de C
Menor distância de D ao objectivo F = 2
h(D) = 2
Menor distância de F ao objectivo F = 0
h(F) = 0


#### Exercício 1
Para não o fazer manualmente, construa uma função para verificar se a heurística do grafo abstracto é admissível.

In [55]:
# Resolução do exercício 1
def is_admissivel(problem, h = None):
    h = h or problem.h
    for estado in problem.grafo.keys():
        p_deste_estado = ProblemaGrafoHs(estado)

        resultado = uniform_cost_search(p_deste_estado)
        if resultado is not None and p_deste_estado.h1(Node(estado)) > resultado.path_cost: # > == not <=
            return False

    return True

is_admissivel(p, p.h1)

True

### Consistência de uma função heurística
Uma heurística é consistente se para todos os arcos X -> Y, a diferença das heurísticas h(X) - h(Y) é menor do que o custo do arco.
**Uma heurística consistente garante a optimalidade do A* em grafo.**

A solução que o A* em grafo devolveu foi a melhor. Foi a melhor por acaso?
Não. Esta heurística é consistente e quando a heurística é consistente, o A* (grafo) garante a melhor solução, de caminho mais curto.
Porque é que a heurística é consistente?
Para que uma heurística seja consistente é preciso que para qualquer arco dirigido x1 -> x2, h(x1)-h(x2) <= c(x1,x2)

No grafo temos os seguintes arcos:
I --> A
I --> B
A --> I
B --> I
B --> D 
B --> F 
A --> C
A --> D
D --> C
D --> F

Verifiquemos a consistência começando pelo arco I --> A
1. h(I)-h(A)=7-6=1   e  c(I,A)=2   ---  1 <= 2    satisfaz
2. h(I)-h(B)=7-2=5   e  c(I,B)=6   ---  5 <= 6    satisfaz
3. h(A)-h(I)=6-7=-1  e  c(A,I)=2   --- -1 <= 2    satisfaz
4. h(B)-h(I)=2-7=-5  e  c(B,I)=6   --- -5 <= 6    satisfaz
5. h(B)-h(D)=2-2=0   e  c(B,D)=1   ---  0 <= 1    satisfaz
6. h(B)-h(F)=2-0=2   e  c(B,F)=5   ---  2 <= 5    satisfaz
7. h(A)-h(C)=6-4=2   e  c(A,C)=2   ---  2 <= 2    satisfaz
8. h(A)-h(D)=6-2=4   e  c(A,D)=4   ---  4 <= 4    satisfaz
9. h(D)-h(C)=2-4=-2  e  c(D,C)=3   --- -2 <= 3    satisfaz
10. h(D)-h(F)=2-0=2  e  c(D,F)=2   ---  2 <= 2    satisfaz

Bastaria violar a regra para um arco para que não fosse consistente, mas isso não acontece.
Conclusão: h1 é consistente, garantindo a optimalidade.

#### Exercício 2
Para não o fazer manualmente, construa uma função para verificar se a heurística do grafo abstracto é consistente. Teste-a com o grafo abstracto inicial.

In [56]:
# Resolução do exercício 2
def is_consistente(problem, h=None):
    h = h or problem.h
    for estado in problem.grafo.keys():
        no_estado = Node(estado)
        expandidos = no_estado.expand(problem)

        h_estado = h(no_estado)
        if any(h_estado - h(expandido) > expandido.path_cost - expandido.parent.path_cost
               for expandido in expandidos):
            return False

    return True

is_consistente(p, p.h1)

True

#### Exercício 3
Crie uma nova heurística h2(x) que seja inconsistente e volte a correr o A* com h2(x). Cuidado que o facto de uma heurística ser inconsistente não impede o A* de encontrar a solução melhor. Não é garantido que a encontre mas pode encontrar. Nesse caso procure uma função h3 que devolva uma solução que não seja a melhor.

In [60]:
# Resolução do exercício 3
def h2(no):
    h = {'I':9, 'A':6,'B':2,'C':4,'D':2,'F':0}
    return h[no.state]

is_consistente(p, h2)

False

## Pacman Grave come pastilha
Relembremos o problema do Pacman Grave que come a pastilha. Vamos formular problemas de navegação sujeita às forças da gravidade, numa grelha 2D em que algumas células são obstáculos impassíveis. O estado vai ser um tuplo com as coordenadas (x,y) e as acções (x+dx,y+dy) correspondem às casas vizinhas, desde que não sejam obstáculos. (dx,dy) corresponde ao deslocamento: (0,1) corresponde ao norte e (-1,-1) a sudoeste.
Notem que as acções para baixo não custam nada, as acções na horizontal custam 1 e as acções para cima custam 3.    

In [46]:
def manhatan(p,q):
    (x1,y1) = p
    (x2,y2) = q
    return abs(x1-x2) + abs(y1-y2)

"""Encontrar um caminho numa grelha 2D com obstáculos. Os obstáculos são células (x, y)."""
class GridProblem(Problem):
    def __init__(self, initial=(1, 1), goal=(7, 8), obstacles=()):
        self.initial=initial
        self.goal=goal 
        self.obstacles=set(obstacles) - {initial, goal}

    directions = {"N":(0, -1), "W":(-1, 0), "E":(1,  0),"S":(0, +1)}  # ortogonais
    
    costs = {"N":3, "W":1, "E":1,"S":0}  # ortogonais
                  
    def result(self, state, action): 
        "Tanto as acções como os estados são representados por pares (x,y)."
        (x,y) = state
        (dx,dy) = self.directions[action]
        return (x+dx,y+dy) 
    
    def actions(self, state):
        """Podes move-te para uma célula em qualquer das direcções para uma casa 
           que não seja obstáculo."""
        x, y = state
        return [act for act in self.directions.keys() if (x+self.directions[act][0],y+self.directions[act][1]) not in self.obstacles]
    
    def path_cost(self,c,state,action,new):
        return c+self.costs[action]
    
    def display(self, state,d):
        """ print the state please"""
        output=""
        for j in range(d):
            for i in range(d):
                if state ==(i,j):
                    ch = '@'
                elif self.goal==(i,j):
                    ch = "*"
                elif (i,j) in self.obstacles:
                    ch = "#"
                else:
                    ch = "."
                output += ch + " "
            output += "\n"
        print(output)
        
    def display_trace(self,d,plan):
        path = set()
        st = self.initial
        for a in plan[:-1]:
            st = self.result(st,a)
            path.add(st)
            
        """ print the state please"""
        output=""
        for j in range(d):
            for i in range(d):
                if self.initial ==(i,j):
                    ch = '@'
                elif self.goal==(i,j):
                    ch = "*"
                elif (i,j) in self.obstacles:
                    ch = "#"
                elif (i,j) in path:
                    ch = '+'
                else:
                    ch = "."
                output += ch + " "
            output += "\n"
        print(output)

    def manhatan_goal(self,no) : 
        """Uma heurística é uma função de um estado.
        Nesta implementação, é uma função do estado associado ao nó
        (objecto da classe Node) fornecido como argumento.
        """
        return manhatan(no.state,self.goal)

def line(x, y, dx, dy, length):
    """Uma linha de células de comprimento 'length' começando em (x, y) na direcção (dx, dy)."""
    return {(x + i * dx, y + i * dy) for i in range(length)}

def quadro(x, y, length):
    """Uma moldura quadrada de células de comprimento 'length' começando no topo esquerdo (x, y)."""
    return line(x,y,0,1,length) | line(x+length-1,y,0,1,length) | line(x,y,1,0,length) | line(x,y+length-1,1,0,length)

l = line(2,2,1,0,6)
c = line(2,3,0,1,4)
fronteira = quadro(0,0,10)
g = GridProblem(obstacles=fronteira | l | c,goal=(3,3))
g.display(g.initial,10)

# teste da função de distância de Manhatan
print('Distância entre (1,4) e (3,7):',manhatan((1,4),(3,7)))

# # # # # # # # # # 
# @ . . . . . . . # 
# . # # # # # # . # 
# . # * . . . . . # 
# . # . . . . . . # 
# . # . . . . . . # 
# . # . . . . . . # 
# . . . . . . . . # 
# . . . . . . . . # 
# # # # # # # # # # 

Distância entre (1,4) e (3,7): 5


#### Exercício 4
Apliquem o "Greedy" e o A* ao problema do Pacman, usando a distância de Manhatan entre dois pontos como função heuristica. Reparem que essa heurística não é nem consistente nem admissível. O A* não terá que devolver a solução de menor custo. Podem comparar o custo com o devolvido pelo custo uniforme e comparar os tempos de execução.

In [49]:
# Resolução do exercício 4
res_gbfs = greedy_best_first_graph_search(g,g.manhatan_goal)
print(res_gbfs.solution())
print(res_gbfs.path_cost)

res_astar = astar_search(g,g.manhatan_goal)
print(res_astar.solution())
print(res_astar.path_cost)

['S', 'S', 'S', 'S', 'S', 'S', 'E', 'E', 'N', 'N', 'N', 'N']
14
['S', 'S', 'S', 'S', 'S', 'S', 'E', 'E', 'N', 'N', 'N', 'N']
14


#### Exercício 5
A heurística da distância de Manhatan entre a posição do Pacman e a posição da pastilha não é admissível nem consistente. Notem que se o Pacman está na casa adjacente a norte da pastilha, o custo menor até lá é 0 e a heurística indica 1, sobreestimando o seu valor e violando a propriedade de admissibilidade.
Desenhe uma heurística consistente e volte a executar o A* e o Greedy. Notem que essa heurística tem de levar em conta os diferentes custos dos movimentos devido ao efeito da força de gravidade sobre o Pacman.

In [67]:
# Resolução do exercício 5
def h(node: Node):
    return 0

es_gbfs = greedy_best_first_graph_search(g, h)
print(res_gbfs.solution())
print(res_gbfs.path_cost)

res_astar = astar_search(g, h)
print(res_astar.solution())
print(res_astar.path_cost)

['S', 'S', 'S', 'S', 'S', 'S', 'E', 'E', 'N', 'N', 'N', 'N']
14
['E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'W', 'W', 'W', 'W', 'W']
12


## Problema do Puzzle 8

![](https://ece.uwaterloo.ca/~dwharder/aads/Algorithms/N_puzzles/images/puz3.png)

Consideremos o puzzle de 8 que é modelizado a seguir em que temos duas funções heurísticas: h1 e h2.

A função heurística h1(x) conta apenas o número de peças bem colocadas. Estar bem colocada é estar na posição desejável ou objetivo (neste exemplo, o objetivo atinge-se com os números ordenados e a casa vazia situada no canto inferior direito). Na configuração da esquerda só a peça 2 está bem colocada e assim é preciso no mínimo deslocar 7 peças. Esta heurística é admissível e consistente.

A função h2(x) devolve a soma total das distâncias de manhatan de cada peça à sua posição objetivo. Na configuração da esquerda:

    A peça 1 está a 2 de distância.
    A peça 2 está a 0 de distância.
    A peça 3 está a 3 de distância.
    A peça 4 está apenas à distância de 1.
    A peça 5 está a 2 de distância.
    A 6 está a 1 de distância.
    A 7 está a 4 e 
    A 8 a 2.
    
    Dá um total de 15 movimentos de distância, muito mais próximo do número mínimo do que o dado por h1.


In [68]:
from itertools import combinations

class EightPuzzle(Problem):
    """ The problem of sliding tiles numbered from 1 to 8 on a 3x3 board,
    where one of the squares is a blank, trying to reach a goal configuration.
    A board state is represented as a tuple of length 9, where the element at index i 
    represents the tile number at index i, or 0 if for the empty square, e.g. the goal:
        1 2 3
        4 5 6 ==> (1, 2, 3, 4, 5, 6, 7, 8, 0)
        7 8 _
    """

    def __init__(self, initial, goal=(0, 1, 2, 3, 4, 5, 6, 7, 8)):
        assert inversions(initial) % 2 == inversions(goal) % 2 # Parity check
        self.initial, self.goal = initial, goal
    
    def actions(self, state):
        """The indexes of the squares that the blank can move to."""
        moves = ((1, 3),    (0, 2, 4),    (1, 5),
                 (0, 4, 6), (1, 3, 5, 7), (2, 4, 8),
                 (3, 7),    (4, 6, 8),    (7, 5))
        blank = state.index(0)
        return moves[blank]
    
    def result(self, state, action):
        """Swap the blank with the square numbered `action`."""
        s = list(state)
        blank = state.index(0)
        s[action], s[blank] = s[blank], s[action]
        return tuple(s)
    
    def h1(self, node):
        """The misplaced tiles heuristic."""
        return hamming_distance(node.state, self.goal) - 1
    
    def h2(self, node):
        """The Manhattan heuristic."""
        X = (0, 1, 2, 0, 1, 2, 0, 1, 2)
        Y = (0, 0, 0, 1, 1, 1, 2, 2, 2)
        return sum(abs(X[s] - X[g]) + abs(Y[s] - Y[g])
                   for (s, g) in zip(node.state, self.goal) if s != 0)
    
    def h(self, node): return self.h2(node)
    
    
def hamming_distance(A, B):
    "Number of positions where vectors A and B are different."
    return sum(a != b for a, b in zip(A, B))
    

def inversions(board):
    "The number of times a piece is a smaller number than a following piece."
    return sum((a > b and a != 0 and b != 0) for (a, b) in combinations(board, 2))
    
    
def board8(board, fmt=(3 * '{} {} {}\n')):
    "A string representing an 8-puzzle board"
    return fmt.format(*board).replace('0', '_')

class Board(defaultdict):
    empty = '.'
    off = '#'
    def __init__(self, board=None, width=8, height=8, to_move=None, **kwds):
        if board is not None:
            self.update(board)
            self.width, self.height = (board.width, board.height) 
        else:
            self.width, self.height = (width, height)
        self.to_move = to_move

    def __missing__(self, key):
        x, y = key
        if x < 0 or x >= self.width or y < 0 or y >= self.height:
            return self.off
        else:
            return self.empty
        
    def __repr__(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(row(y) for y in range(self.height))
            
    def __hash__(self): 
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

Temos alguns exemplos de puzzles

In [69]:
e1 = EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8))
e2 = EightPuzzle((1, 2, 3, 4, 5, 6, 7, 8, 0))
e3 = EightPuzzle((4, 0, 2, 5, 1, 3, 7, 8, 6))
e4 = EightPuzzle((7, 2, 4, 5, 0, 6, 8, 3, 1))
e5 = EightPuzzle((8, 6, 7, 2, 5, 4, 3, 0, 1))

Testemos as heurísticas para cada um dos casos

In [70]:
print('Goal =')
print(board8(e1.goal))
print('Teste das heuristicas para cada um dos casos:')
print()
for e in [e1, e2, e3, e4]:
    print(board8(e.initial))
    print('Número de peças mal colocadas:',e.h1(Node(e.initial)))
    print('Distância total à solução:', e.h2(Node(e.initial)))

Goal =
_ 1 2
3 4 5
6 7 8

Teste das heuristicas para cada um dos casos:

1 4 2
_ 7 5
3 6 8

Número de peças mal colocadas: 5
Distância total à solução: 5
1 2 3
4 5 6
7 8 _

Número de peças mal colocadas: 8
Distância total à solução: 12
4 _ 2
5 1 3
7 8 6

Número de peças mal colocadas: 7
Distância total à solução: 11
7 2 4
5 _ 6
8 3 1

Número de peças mal colocadas: 8
Distância total à solução: 18


Vamos aplicar o A* com as duas heurísticas ao primeiro problema: e1

In [114]:
import timeit

print('A* com a heurística do número de peças mal colocadas')
start = timeit.default_timer()
res_astar = astar_search(e1,e1.h1)
stop = timeit.default_timer()
timeGraph = stop-start
print('Time:', timeGraph)
print("Solução:",res_astar.solution())
print('\nA* com a heurística do somatório das distâncias de cada peça ao objetivo')
start = timeit.default_timer()
res_astar = astar_search(e1,e1.h2)
stop = timeit.default_timer()
timeGraph = stop-start
print('Time:', timeGraph)
print("Solução:",res_astar.solution()) # corresponde aos movimentos da casa vazia

A* com a heurística do número de peças mal colocadas
Time: 0.00032434999957331456
Solução: [6, 7, 4, 1, 0]

A* com a heurística do somatório das distâncias de cada peça ao objetivo
Time: 0.0003239220000068599
Solução: [6, 7, 4, 1, 0]


Para vizualizarmos a sequência de estados que corresponde à solução vamos definir a função ***path_states()*** que devolve todos os estados ao longo de um node.

In [115]:
def path_states(node):
    "The sequence of states to get to this node."
    if node == None: 
        return []
    return path_states(node.parent) + [node.state]

Vamos aplicá-la

In [116]:
for s in path_states(res_astar):
    print(board8(s))

1 4 2
_ 7 5
3 6 8

1 4 2
3 7 5
_ 6 8

1 4 2
3 7 5
6 _ 8

1 4 2
3 _ 5
6 7 8

1 _ 2
3 4 5
6 7 8

_ 1 2
3 4 5
6 7 8



Este puzzle é demasiado simples para podermos apreciar e comparar o impacto que as duas heurísticas têm sobre o A*. Vamos ao segundo puzzle, em e2

In [117]:
print('A* com a heurística do número de peças mal colocadas')
start = timeit.default_timer()
res_astar = astar_search(e2,e2.h1)
stop = timeit.default_timer()
timeGraph = stop-start
print('Time:', timeGraph)
print("Solução:",res_astar.solution())
print('\nA* com a heurística do somatório das distâncias de cada peça ao objectivo')
start = timeit.default_timer()
res_astar = astar_search(e2,e2.h2)
stop = timeit.default_timer()
timeGraph = stop-start
print('Time:', timeGraph)
print("Solução:",res_astar.solution())

A* com a heurística do número de peças mal colocadas
Time: 10.7498236739998
Solução: [5, 4, 3, 6, 7, 8, 5, 2, 1, 4, 7, 6, 3, 4, 5, 8, 7, 6, 3, 4, 1, 0]

A* com a heurística do somatório das distâncias de cada peça ao objectivo
Time: 0.19306565499982753
Solução: [5, 4, 3, 6, 7, 8, 5, 2, 1, 4, 7, 6, 3, 4, 5, 8, 7, 6, 3, 4, 1, 0]


In [118]:
for s in path_states(res_astar):
    print(board8(s))

1 2 3
4 5 6
7 8 _

1 2 3
4 5 _
7 8 6

1 2 3
4 _ 5
7 8 6

1 2 3
_ 4 5
7 8 6

1 2 3
7 4 5
_ 8 6

1 2 3
7 4 5
8 _ 6

1 2 3
7 4 5
8 6 _

1 2 3
7 4 _
8 6 5

1 2 _
7 4 3
8 6 5

1 _ 2
7 4 3
8 6 5

1 4 2
7 _ 3
8 6 5

1 4 2
7 6 3
8 _ 5

1 4 2
7 6 3
_ 8 5

1 4 2
_ 6 3
7 8 5

1 4 2
6 _ 3
7 8 5

1 4 2
6 3 _
7 8 5

1 4 2
6 3 5
7 8 _

1 4 2
6 3 5
7 _ 8

1 4 2
6 3 5
_ 7 8

1 4 2
_ 3 5
6 7 8

1 4 2
3 _ 5
6 7 8

1 _ 2
3 4 5
6 7 8

_ 1 2
3 4 5
6 7 8



Não aconselhamos a tentarem aplicar a profundidade primeiro a este problema mas podem tentar descomentando o código a seguir

In [ ]:

# print('Profundidade primeiro em grafo')
# start = timeit.default_timer()
# res_prof = depth_first_graph_search(e1)
# stop = timeit.default_timer()
# timeGraph = stop-start
# print('Time: ', timeGraph)
# print("Solução:",res_prof.solution())


Vamos aplicar o "Greedy" para o problema e1

In [119]:
print('Greedy com a heurística do número de mal colocados')
start = timeit.default_timer()
res_greedy_h1 = greedy_best_first_graph_search(e1,e1.h1)
stop = timeit.default_timer()
timeGraph = stop-start
print('Time:', timeGraph)
print("Solução:",res_greedy_h1.solution())
print('\nGreedy com a heurística da soma total de Manhatan')
start = timeit.default_timer()
res_greedy_h2 = greedy_best_first_graph_search(e1,e1.h2)
stop = timeit.default_timer()
timeGraph = stop-start
print('Time:', timeGraph)
print("Solução:",res_greedy_h2.solution())

Greedy com a heurística do número de mal colocados
Time: 0.0003100819999417581
Solução: [6, 7, 4, 1, 0]

Greedy com a heurística da soma total de Manhatan
Time: 0.00034219000008306466
Solução: [6, 7, 4, 1, 0]


E agora para o segundo problema

In [120]:
print('Greedy com a heurística do número de mal colocados')
start = timeit.default_timer()
res_greedy_h1 = greedy_best_first_graph_search(e2,e2.h1)
stop = timeit.default_timer()
timeGraph = stop-start
print('Time:', timeGraph)
print("Solução:",res_greedy_h1.solution())
print('\nGreedy com a heurística da soma total de Manhatan')
start = timeit.default_timer()
res_greedy_h2 = greedy_best_first_graph_search(e2,e2.h2)
stop = timeit.default_timer()
timeGraph = stop-start
print('Time:', timeGraph)
print("Solução:",res_greedy_h2.solution())

Greedy com a heurística do número de mal colocados
Time: 0.05406351800002085
Solução: [7, 6, 3, 4, 5, 2, 1, 0, 3, 6, 7, 4, 3, 6, 7, 4, 3, 0, 1, 2, 5, 4, 7, 8, 5, 2, 1, 0, 3, 4, 5, 8, 7, 4, 5, 8, 7, 4, 3, 6, 7, 8, 5, 4, 7, 6, 3, 0, 1, 4, 3, 0, 1, 4, 5, 2, 1, 0, 3, 4, 1, 0, 3, 4, 5, 2, 1, 0, 3, 4, 1, 2, 5, 4, 3, 0, 1, 4, 5, 2, 1, 0]

Greedy com a heurística da soma total de Manhatan
Time: 0.04414541000005556
Solução: [5, 2, 1, 4, 5, 2, 1, 0, 3, 4, 7, 8, 5, 4, 7, 6, 3, 4, 5, 8, 7, 4, 3, 6, 7, 8, 5, 2, 1, 0, 3, 4, 1, 2, 5, 8, 7, 6, 3, 0, 1, 2, 5, 4, 1, 2, 5, 4, 7, 8, 5, 2, 1, 4, 5, 8, 7, 4, 1, 2, 5, 4, 7, 8, 5, 2, 1, 0]


O "Greedy" teve um bom desempenho mesmo no problema mais duro, como iremos ver quando compararmos com a procura em largura, não informada.

In [121]:
print('Largura em grafo, problema e1')
start = timeit.default_timer()
res_breadth = breadth_first_graph_search(e1)
stop = timeit.default_timer()
timeGraph = stop-start
print('Time:', timeGraph)
print("Solução:",res_breadth.solution())
print('\nLargura em Grafo, problema e2')
start = timeit.default_timer()
res_breadth = breadth_first_graph_search(e2)
stop = timeit.default_timer()
timeGraph = stop-start
print('Time:', timeGraph)
print("Solução:",res_breadth.solution())

Largura em grafo, problema e1
Time: 0.0011960780002482352
Solução: [6, 7, 4, 1, 0]

Largura em Grafo, problema e2
Time: 386.3912395790003
Solução: [7, 4, 5, 2, 1, 0, 3, 4, 5, 2, 1, 4, 7, 6, 3, 0, 1, 2, 5, 4, 3, 0]


Vamos comparar a procura em largura com o aprofundamento progressivo, diferença que será mais notória no problema mais difícil dos dois.

In [123]:
print('Progressivo em grafo, problema e1')
start = timeit.default_timer()
res_prog = iterative_deepening_graph_search(e1)
stop = timeit.default_timer()
timeGraph = stop-start
print('Time:', timeGraph)
print("Solução:",res_prog.solution())
print('\nProgressivo em Grafo, problema e2')
start = timeit.default_timer()
res_prog = iterative_deepening_graph_search(e2)
stop = timeit.default_timer()
timeGraph = stop-start
print('Time:', timeGraph)
print("Solução:",res_prog.solution())


Progressivo em grafo, problema e1
Time: 0.0003327299996271904
Solução: [6, 7, 4, 1, 0]

Progressivo em Grafo, problema e2
Time: 0.5136134829999719
Solução: [5, 4, 3, 6, 7, 8, 5, 4, 1, 2, 5, 4, 1, 2, 5, 4, 7, 6, 3, 4, 5, 8, 7, 6, 3, 4, 1, 0]


#### Exercício 6
Considere o problema da inversão de setas que está definido em Python já a seguir. Desenvolva duas funções heurísticas:
1. A primeira calcula a menor das distâncias de um estado aos dois estados objectivos. A distância entre dois estados devolve o número de setas correspondentes com orientações diferentes. Por exemplo, a distância entre [d,d,d,e,e,e] e [e,d,e,d,e,d] é igual a 4. Só as duas segundas setas e as duas quintas setas é que possuem a mesma orientação.
2. Número de pares adjacentes de setas com a mesma orientação. O estado inicial: [d,d,d,e,e,e] possui um nº de pares adjacentes com a mesma orientação que é igual a 4. Os dois objectivos têm ambos 0 como valor heurístico.

In [131]:
class EstadoSetas :

    """Um estado do problema da inversao das setas
        Uma lista de 6 setas (e's ou d's), indicando para cada seta se está orientada
        para a esquerda para para a direita
        A ordem da esquerda para a direita corresponde às setas de cima para baixo
    """
    def __init__(self,setas = ["e","e","e","d","d","d"]) :
        self.setas = setas

        
    def flip(self,seta) :
        """ Inversão do sentido de uma seta: de e para d e de d para e"""
        if seta=="e":
            return "d"
        else:
            return "e"

    def inverte(self,n) :
        """ Inverte duas setas, na posição n e n+1. A primeira seta está na posição 0
            e para inverter a primeira e a segunda, de cima para baixo, implica n = 0.
            Gera uma nova lista.
        """
        copye = []
        for i in range(len(self.setas)) :
            if i == n-1 or i == n :
                copye.append(self.flip(self.setas[i]))
            else:
                copye.append(self.setas[i])
        return EstadoSetas(copye)
        
    def __str__(self) :
        return str(self.setas)
    
    def __eq__(self,estado) :
        """Definir em que circunstância os dois estados são considerados iguais.
        Necessário para os algoritmos de procura em grafo.
        """
        return self.setas == estado.setas
    
    def __lt__(self,estado):
            return True

    def __hash__(self) :
        """Necessário para os algoritmos de procura em grafo."""
        return hash((str(self.setas)))



class ProblemaSetas(Problem) :
    
    def __init__(self,initial = EstadoSetas(["e","e","e","d","d","d"])) :
        super().__init__(initial)

    
    def actions(self,estado) :
        """ A acção 1 corresponde a inverter as setas de índices 0 e 1 da lista
            A acção 5 coorresponde a inverter as setas de índices 4 e 5, as últimas duas """
        accoes = [1,2,3,4,5]
        
        return accoes 

    def result(self,estado,acao) :
        if acao in self.actions(estado) :
            resultante = estado.inverte(acao)
        else:
            raise "Há aqui qualquer coisa mal>> acao não reconhecida"
 
        return resultante

In [130]:
# Resolução do exercício 6
psetas = ProblemaSetas()

def h1(node: Node):
    """A menor das distâncias de um estado aos dois estados objectivos. A distância entre dois estados devolve o número de setas correspondentes com orientações diferentes. Por exemplo, a distância entre [d,d,d,e,e,e] e [e,d,e,d,e,d] é igual a 4. Só as duas segundas setas e as duas quintas setas é que possuem a mesma orientação."""
    pass

def h2(node: Node):
    """Número de pares adjacentes de setas com a mesma orientação. O estado inicial: [d,d,d,e,e,e] possui um nº de pares adjacentes com a mesma orientação que é igual a 4. Os dois objectivos têm ambos 0 como valor heurístico."""
    cnt = 0

    for i in range(len(node.state.setas) - 1):
        if node.state.setas[i] == node.state.setas[i+1]:
            cnt += 1

    return cnt

no = Node(psetas.initial)
print('H1', psetas.initial, h1(no))
print('H2', psetas.initial, h2(no))

H1 ['e', 'e', 'e', 'd', 'd', 'd'] None
H2 ['e', 'e', 'e', 'd', 'd', 'd'] 4
